TODO
* index_oper (create 0/1 is the certain for top operators, something like this)
* do with name_mfi as well

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

In [2]:
%cd ../

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303


In [3]:
df = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
test_ids = deepcopy(test_data.id)

/tmp/ipykernel_35665/2838075050.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/train.csv")
/tmp/ipykernel_35665/2838075050.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv("data/test.csv")


In [4]:
df.head(3)

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,1043_-1,628629.0,Участок,7503.0,N,0.0,Y,N,87.0,...,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,1023_-1,102976.0,ММПО,7503.0,N,0.0,N,N,107.0,...,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,1018_-1,620962.0,Цех,7503.0,N,0.0,Y,N,50.0,...,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0


In [5]:
X = df.drop(columns=["label"])
y = df.label
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

del [[X, df]]
gc.collect()
X=pd.DataFrame()
df=pd.DataFrame()

In [6]:
df.head(3)

""


In [7]:
cat_features = [
    "type", "is_privatecategory", "class", "is_in_yandex", "is_return", "mailtype", "mailctg", "directctg",
    "oper_type", "oper_attr"
]

def prepare_data(df):
    
    # 1. split oper type and attr
    df["oper_type"], df["oper_attr"] = zip(*df["oper_type + oper_attr"].map(lambda text: text.split("_")))
    df.drop(columns=["oper_type + oper_attr"], inplace=True)
    
    # 2. delete ids and so on
    df.drop(columns=["id", "index_oper", "name_mfi"], inplace=True)
    
    # 3. delete low variant columns
    df.drop(columns=["mailrank", "postmark"], inplace=True)
    
    # 4. delete suspicious
    df.drop(columns=["priority", "total_qty_oper_login_1", "total_qty_oper_login_0", "total_qty_over_index_and_type", "total_qty_over_index"], inplace=True)
 
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))

    return df

In [ ]:
X_train = prepare_data(X_train)
X_val = prepare_data(X_val)
test_df = prepare_data(test_data)

In [ ]:
X_train.head()

In [ ]:
model = CatBoostClassifier(random_state=0, max_depth=5, verbose=10, iterations=200, auto_class_weights="Balanced", cat_features=cat_features)

In [ ]:
model.fit(X_train, y_train)

In [12]:
def score_model(model, X, y):
    pred = model.predict(X)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    tpr = (tp) / (tp + fn)
    fpr = (fp) / (fp + tn)
    
    r_score = (tp) / (tp + fn)
    ra_score = (1 + tpr - fpr) / 2
    
    print(tn, fp, fn, tp, r_score, ra_score)
    
    score = 0.1 * r_score + 0.9 * ra_score
    
    return score

In [13]:
score_model(model, X_val, y_val)

1115122 50831 146 33901 0.9957118101447998 0.9760578566090399


0.978023251962616

In [14]:
test_preds = model.predict(test_df)

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303/venv/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [15]:
submission = pd.DataFrame({'id': test_ids, 'label': test_preds})

In [16]:
submission.to_csv("submission.csv", index=False)

In [17]:
test_preds.mean()

0.07070675